In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Fixed typo "self,hidden_size"

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  
        return out

In [3]:
def train_lstm_model(model, X_train, y_train, epochs=50, batch_size=32, learning_rate=0.001):
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train)
    
    # Create dataset and dataloader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    # Loss function and optimizer
    criterion = nn.MSELoss()  # For regression tasks
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print progress
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}')
    
    print('Training complete')

In [4]:

if __name__ == "__main__":
    # Example parameters
    input_size = 10  
    hidden_size = 64 
    output_size = 1  
    num_layers = 2   
    
    
    model = LSTMModel(input_size, hidden_size, output_size, num_layers)
    
    import numpy as np
    
    X_train = np.random.randn(100, 20, input_size)  
    y_train = np.random.randn(100, output_size)    
    
    # Train the model
    train_lstm_model(model, X_train, y_train, epochs=50)
    
    # Example of making predictions
    model.eval()
    with torch.no_grad():
        X_test = torch.FloatTensor(np.random.randn(10, 20, input_size))
        predictions = model(X_test)
        print("Predictions shape:", predictions.shape)

Epoch [10/50], Loss: 0.7233
Epoch [20/50], Loss: 0.8242
Epoch [30/50], Loss: 0.4234
Epoch [40/50], Loss: 0.2387
Epoch [50/50], Loss: 0.1881
Training complete
Predictions shape: torch.Size([10, 1])


In [5]:
predictions

tensor([[-0.5807],
        [-1.8266],
        [ 0.9912],
        [ 1.6866],
        [ 0.5765],
        [ 0.6393],
        [ 0.9084],
        [ 0.0583],
        [ 0.7286],
        [-0.0510]])

In [1]:
from graphviz import Digraph

def create_lstm_flowchart():
    dot = Digraph(format='png')
    
    # Nodes
    dot.node('x_t', 'Input (x_t)')
    dot.node('h_t-1', 'Previous Hidden State (h_{t-1})')
    dot.node('C_t-1', 'Previous Cell State (C_{t-1})')
    dot.node('f_t', 'Forget Gate\nσ(W_f[x_t, h_{t-1}] + b_f)')
    dot.node('i_t', 'Input Gate\nσ(W_i[x_t, h_{t-1}] + b_i)')
    dot.node('C_tilda', 'Candidate Memory\ntanh(W_c[x_t, h_{t-1}] + b_c)')
    dot.node('C_t', 'Current Cell State\nf_t * C_{t-1} + i_t * C_tilda')
    dot.node('o_t', 'Output Gate\nσ(W_o[x_t, h_{t-1}] + b_o)')
    dot.node('h_t', 'New Hidden State\no_t * tanh(C_t)')
    
    # Edges
    dot.edges(['x_th_t-1', 'h_t-1f_t', 'h_t-1i_t', 'h_t-1C_tilda', 'h_t-1o_t'])
    dot.edge('x_t', 'f_t')
    dot.edge('x_t', 'i_t')
    dot.edge('x_t', 'C_tilda')
    dot.edge('x_t', 'o_t')
    dot.edge('f_t', 'C_t', label=' * C_{t-1}')
    dot.edge('i_t', 'C_t', label=' * C_tilda')
    dot.edge('C_tilda', 'C_t')
    dot.edge('C_t', 'h_t', label='tanh')
    dot.edge('o_t', 'h_t', label=' *')
    
    return dot

# Generate flowchart
lstm_flowchart = create_lstm_flowchart()
lstm_flowchart.render('lstm_flowchart', format='png', cleanup=False)


ModuleNotFoundError: No module named 'graphviz'

In [1]:
!pip install graphviz

  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
